# Criando uma Árvore de Decisão

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.model_selection import StratifiedShuffleSplit
from sklearn import tree
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier


In [ ]:
dados = pd.read_csv('data/creditcard.csv')
dados.head()

### O número de: transação, fraudes e dados normais

In [ ]:
dados['Class'].unique()

In [ ]:
n_transacoes = dados['Class'].count()
n_fraudes = dados['Class'].sum()
n_normais = n_transacoes - n_fraudes

fraudes_porc = ((n_fraudes / n_transacoes) *100)
normais_porc = (n_normais / n_transacoes) *100


print("Número de transações: ", n_transacoes)
print("Número de fraudes: ", n_fraudes, ", ", "%.2f" %fraudes_porc, "%", sep = "")
print("Número de fraudes: ", n_normais, ", ", "%.2f" %normais_porc, "%", sep = "")

In [ ]:
x = dados.drop('Class', axis = 1).values
y = dados['Class'].values

## Funções

### Criando uma árvore de decisão com o scikit-learn

In [ ]:
def executar_validador(x, y):
    # Vamos garantir que o meu conjunto vai ter 10%
    # das transações de cada uma das classes.
    validador = StratifiedShuffleSplit(n_splits=1,
                                   test_size=0.1, random_state=0)
    validador.split(x, y)
    for treino_id, teste_id in validador.split(x, y):
        x_train, x_test = x[treino_id], x[teste_id]
        y_train, y_test = y[treino_id], y[teste_id]
    return x_train, x_test, y_train, y_test

### Gerando o modelo 

In [ ]:
def executar_classificador(classificador, x_train, x_test, y_train):
    arvore = classificador.fit(x_train, y_train)
    y_pred = arvore.predict(x_test)
    return y_pred

### Salvando uma imagem da árvore

In [ ]:
def salvar_arvore(classificador, nome):
    plt.figure(figsize=(200,100))
    tree.plot_tree(classificador, filled=True, fontsize=14)
    plt.savefig(nome)
    plt.close()

### Medindo a acurácia da árvore

In [ ]:
def validar_arvore(y_test, y_pred):
    print("accuracy_score: ", accuracy_score(y_test, y_pred))
    print("precision_score: ", precision_score(y_test, y_pred))
    print("recall_score: ", recall_score(y_test, y_pred))
    print("confusion_matrix: \n", confusion_matrix(y_test, y_pred))
    

## EXECUTANDO

### Execução do validador

In [ ]:
x = dados.drop('Class', axis = 1).values
y = dados['Class'].values
x_train, x_test, y_train, y_test = executar_validador(x, y)

### Execução do classificador `DecisionTreeClassifier()`

In [ ]:
classificador_arvore_decisao = tree.DecisionTreeClassifier()
y_pred_arvore_decisao = executar_classificador(
    classificador_arvore_decisao, x_train, x_test, y_train)

### Criando a figura da árvore de decisão

In [ ]:
salvar_arvore(classificador_arvore_decisao, 'arvore_decisao1.png')

### validacao da árvore de decisão


In [ ]:
validar_arvore(y_test, y_pred_arvore_decisao)

<img src="data/Aula3_video2_imagem1.png">

### Verificando os parâmetros

In [ ]:
print("Parâmetros: \n", classificador_arvore_decisao.get_params())

print("\nProfundidade:", classificador_arvore_decisao.get_depth())

### Limitando a profundidade da árvore em 10

In [ ]:
classificador_arvore_decisao_2 = tree.DecisionTreeClassifier(max_depth=10, 
                                                           random_state=0)
y_pred_arvore_decisao_2 = executar_classificador(
    classificador_arvore_decisao_2, x_train, x_test, y_train)

In [ ]:
validar_arvore(y_test, y_pred_arvore_decisao_2)

### Set o número mínimo de 10 instâncias em cada nó

In [ ]:
classificador_arvore_decisao_3 = tree.DecisionTreeClassifier(max_depth=10,
                                                             min_samples_leaf=10,
                                                           random_state=0)
y_pred_arvore_decisao_3 = executar_classificador(
    classificador_arvore_decisao_3, x_train, x_test, y_train)

In [ ]:
validar_arvore(y_test, y_pred_arvore_decisao_3)

### Das características que vimos e dessas mudanças que fizemos, o que funcionou o melhor para nós e para nossa árvore foi determinar somente a profundidade máxima igual a 10. 

In [ ]:
classificador_arvore_decisao_4 = tree.DecisionTreeClassifier(max_depth=5,
                                                             random_state=0)
y_pred_arvore_decisao_4 = executar_classificador(
    classificador_arvore_decisao_4, x_train, x_test, y_train)

In [ ]:
validar_arvore(y_test, y_pred_arvore_decisao_4)

## Bagging e o Random Forest

### Executando o Random Forest com o scikit-learn

In [ ]:
%%time
cassificador_random_forest = RandomForestClassifier(n_estimators=100, random_state=0)

y_pred_random_forest = executar_classificador (cassificador_random_forest, x_train, x_test, y_train)

In [ ]:
# salvar_arvore(classificador_random_forest.estimators_[0], "random_forest1")
# salvar_arvore(classificador_random_forest.estimators_[1], "random_forest2")

In [ ]:
validar_arvore(y_test, y_pred_random_forest)

In [ ]:
%%time
cassificador_random_forest_2 = RandomForestClassifier(n_estimators=50,max_depth=10, random_state=0)

y_pred_random_forest_2 = executar_classificador (cassificador_random_forest_2, x_train, x_test, y_train)

In [ ]:
validar_arvore(y_test, y_pred_random_forest_2)

## AdaBoost

In [ ]:
%%time
classificador_adaboost = AdaBoostClassifier(random_state=0)
y_pred_adaboost = executar_classificador(classificador_adaboost, x_train, x_test, y_train)

In [ ]:
validar_arvore(y_test, y_pred_adaboost)

In [ ]:
%%time
classificador_adaboost2 = AdaBoostClassifier(random_state=0, n_estimators=100)
y_pred_adaboost2 = executar_classificador(classificador_adaboost2, x_train, x_test, y_train)

In [ ]:
validar_arvore(y_test, y_pred_adaboost2)

In [ ]:
%%time
classificador_adaboost3 = AdaBoostClassifier(random_state=0, n_estimators=200)
y_pred_adaboost3 = executar_classificador(classificador_adaboost3, x_train, x_test, y_train)

In [ ]:
validar_arvore(y_test, y_pred_adaboost3)